In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('dog_food_tree').getOrCreate()
df = spark.read.csv('dog_food.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [2]:
df.show(2)

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
+---+---+----+---+-------+
only showing top 2 rows



In [4]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['A', 'B', 'C', 'D'], outputCol = 'features')

In [7]:
output = assembler.transform(df)

In [8]:
output.show(3)

+---+---+----+---+-------+------------------+
|  A|  B|   C|  D|Spoiled|          features|
+---+---+----+---+-------+------------------+
|  4|  2|12.0|  3|    1.0|[4.0,2.0,12.0,3.0]|
|  5|  6|12.0|  7|    1.0|[5.0,6.0,12.0,7.0]|
|  6|  2|13.0|  6|    1.0|[6.0,2.0,13.0,6.0]|
+---+---+----+---+-------+------------------+
only showing top 3 rows



In [9]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol = 'Spoiled', featuresCol = 'features')

In [10]:
final_df = output.select('features', 'Spoiled')

In [11]:
final_df.show(3)

+------------------+-------+
|          features|Spoiled|
+------------------+-------+
|[4.0,2.0,12.0,3.0]|    1.0|
|[5.0,6.0,12.0,7.0]|    1.0|
|[6.0,2.0,13.0,6.0]|    1.0|
+------------------+-------+
only showing top 3 rows



In [12]:
rf_model = rf.fit(final_df)

In [13]:
final_df.take(1)

[Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)]

In [14]:
rf_model.featureImportances

SparseVector(4, {0: 0.0212, 1: 0.0166, 2: 0.9373, 3: 0.025})